In [1]:
from datetime import date
from pathlib import Path

import aiohttp
import aiomoex
import pandas as pd
from aiomoex import client
from aiomoex.request_helpers import make_url, get_table, make_query
import datatable as dt
from datatable import f

from glob import glob
import seaborn as sns

from parsing.commodities import get_brent_history, __COMMODITIES_FOLDER__

In [2]:
PREFIX = 'BR'
MONTHS = ('F', 'G', 'H', 'J', 'K', 'M', 'N', 'Q', 'U', 'V', 'X', 'Z')
YEARS = range(0, 10)

In [4]:
for month in MONTHS:
    for year in YEARS:
        security = PREFIX + month + str(year)

        url = make_url(engine="futures", market="forts", board="RFUD", security=security, history=True)
        query = make_query(start='2016-01-01', end='2022-01-01')

        async with aiohttp.ClientSession() as session:
            iss = client.ISSClient(session, url, query)

            data = await iss.get_all()
            history = get_table(data, "history")
            if len(history) == 0:
                continue

            bond_df = pd.DataFrame(history)
            bond_df.to_csv(str(__COMMODITIES_FOLDER__.joinpath('history', security + '.csv')), header=True, index=False)


In [5]:
DT = get_brent_history()

In [6]:
DT.to_pandas()

,BOARDID,TRADEDATE,SECID,OPEN,LOW,HIGH,CLOSE,OPENPOSITIONVALUE,VALUE,VOLUME,OPENPOSITION
0,RFUD,2019-09-25,BRV0,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN
1,RFUD,2019-09-26,BRV0,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN
2,RFUD,2019-09-27,BRV0,59.13,59.13,59.13,59.13,1.509958e+05,7.611580e+04,2.0,4.0
3,RFUD,2019-09-30,BRV0,59.40,56.79,59.40,56.79,1.512868e+05,7.539035e+04,2.0,4.0
4,RFUD,2019-10-01,BRV0,NaN,NaN,NaN,NaN,1.507763e+05,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...
17726,RFUD,2016-10-27,BRX6,50.09,49.66,50.77,50.60,1.407277e+10,6.115958e+10,1938058.0,443120.0
17727,RFUD,2016-10-28,BRX6,50.69,49.91,50.88,49.97,1.425438e+10,4.075412e+10,1286571.0,453356.0
17728,RFUD,2016-10-31,BRX6,50.00,48.05,50.13,48.16,7.888373e+09,4.041177e+10,1292924.0,258312.0
17729,RFUD,2016-11-01,BRX6,48.22,47.96,48.34,48.32,0.000000e+00,9.307155e+09,305863.0,0.0
